In [1]:
import os
import json

hfcache = '/mnt/data1/prasann/latticegen/lattice-generation/hfcache'
os.environ['TRANSFORMERS_CACHE'] = hfcache

In [2]:
os.listdir('./T0_explore_data')

['reverse_meaning_data.txt',
 'reverse_sent_data.txt',
 '.ipynb_checkpoints',
 'simplification_data.txt',
 'infilling_data.txt',
 'informal_data.txt',
 'easy_hand_data.txt']

In [3]:

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

tokenizer = AutoTokenizer.from_pretrained("bigscience/T0_3B", cache_dir=hfcache)
model = AutoModelForSeq2SeqLM.from_pretrained("bigscience/T0_3B", cache_dir=hfcache)


2022-07-11 04:29:45.121379: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-11 04:29:45.121411: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [35]:
BASE = './T0_explore_data/'

def read_input_data (fname):
    with open(BASE+fname+".txt") as file:
        lines = file.readlines()
        lines = [line.rstrip() for line in lines]
    while("" in lines) :
        lines.remove("")
    
    return lines

infill_data = read_input_data('infilling_data')
informal_data = read_input_data('informal_data')
revmean_data = read_input_data('reverse_meaning_data')
revsent_data = read_input_data('reverse_sent_data')
simplif_data = read_input_data('simplification_data')
easy_data = read_input_data('easy_hand_data')


In [19]:
inf_prompts = [
    "[TEXT] What goes in the _ ?",
    "Task: Fill in the _ in the following sentence. [TEXT]",
    "Task: replace [blank] with the correct words. [TEXT]",
]


sent_prompts = [
    "Task: what's the opposite of this sentence? [TEXT]",
    "Copy but say the opposite: [TEXT]",
    "[TEXT] What is the opposite of this sentence?",
    "Task: copy but say the opposite emotion. [TEXT]",
    "Task: copy but say the opposite. [TEXT]", #
]

In [20]:

def get_T0_output(prompt, raw):
    inputstring = get_inputstr(prompt, raw)
    inputs = tokenizer.encode(inputstring, return_tensors="pt")
    outputs = model.generate(inputs, max_length=50, num_beams=9, num_return_sequences=9, early_stopping=True, output_scores=True, return_dict_in_generate=True)
    res = []
    #print(outputs)
    for i in range(0, len(outputs.sequences)):
        tmp = {}
        tmp['out_toks'] = tokenizer.decode(outputs.sequences[i])
        tmp['out_sco'] = outputs.sequences_scores[i]
        res.append(tmp)
        print(tmp)
    out = {}
    out['data'] = res
    out['input'] = inputstring
    return out
        
def get_inputstr(prompt, raw):
    return prompt.replace("[TEXT]", raw)

alldat = []

def run_all_data (promptlist, data, label):
    global alldat
    alldata = []
    for p in promptlist:
        curtmp = {}
        curtmp['prompt'] = p
        curtmp['examples'] = []
        for d in data:
            curtmp['examples'].append(get_T0_output(p, d))
        alldata.append(curtmp)
    alldat = alldata
    for a in alldata:
        for e in a['examples']:
            for d in e['data']:
                d['out_sco'] = float(d['out_sco'])
    alldatajson = {}
    alldatajson['data'] = alldata
    with open('./prompt_output/'+label+'.txt', 'w') as f:
        json.dump(alldatajson, f, ensure_ascii=False)



In [34]:

alldatajson = {}
alldatajson['data'] = alldat
with open('./prompt_output/'+'opposite_prompt'+'.json', 'w') as f:
    json.dump(alldatajson, f, ensure_ascii=False)

In [ ]:
run_all_data(sent_prompts, easy_data, 'opposite_prompt')

In [15]:
prompt = sent_prompts[4]
inp = easy_data[5]
inputstr = get_inputstr(prompt, inp)
print(inputstr)
get_T0_output(prompt, inp)

Task: copy but say the opposite. It was a fun experience!
{'out_toks': '<pad> It was a fun experience!</s>', 'out_sco': tensor(-0.3170)}
{'out_toks': '<pad> It was a fun experience</s><pad>', 'out_sco': tensor(-0.8022)}
{'out_toks': '<pad> It was a funny experience!</s>', 'out_sco': tensor(-0.9081)}
{'out_toks': '<pad> <unk>file_gif></s>', 'out_sco': tensor(-0.9201)}
{'out_toks': '<pad> It was a great experience!</s>', 'out_sco': tensor(-0.9259)}
{'out_toks': '<pad> <unk>file_other></s><pad>', 'out_sco': tensor(-0.9371)}
{'out_toks': '<pad> <unk>file_photo></s><pad>', 'out_sco': tensor(-0.9612)}
{'out_toks': '<pad> It was fun experience!</s><pad><pad>', 'out_sco': tensor(-1.2185)}
{'out_toks': '<pad> It was fun!</s><pad><pad><pad>', 'out_sco': tensor(-1.3179)}


{'data': [{'out_toks': '<pad> It was a fun experience!</s>',
   'out_sco': tensor(-0.3170)},
  {'out_toks': '<pad> It was a fun experience</s><pad>',
   'out_sco': tensor(-0.8022)},
  {'out_toks': '<pad> It was a funny experience!</s>',
   'out_sco': tensor(-0.9081)},
  {'out_toks': '<pad> <unk>file_gif></s>', 'out_sco': tensor(-0.9201)},
  {'out_toks': '<pad> It was a great experience!</s>',
   'out_sco': tensor(-0.9259)},
  {'out_toks': '<pad> <unk>file_other></s><pad>', 'out_sco': tensor(-0.9371)},
  {'out_toks': '<pad> <unk>file_photo></s><pad>', 'out_sco': tensor(-0.9612)},
  {'out_toks': '<pad> It was fun experience!</s><pad><pad>',
   'out_sco': tensor(-1.2185)},
  {'out_toks': '<pad> It was fun!</s><pad><pad><pad>',
   'out_sco': tensor(-1.3179)}],
 'input': 'Task: copy but say the opposite. It was a fun experience!'}

In [44]:
#print(outputs)


<pad> The dry sand was hard to grasp.</s><pad>
tensor(-0.1610)
<pad> The wet sand was hard to grasp.</s>
tensor(-0.2598)
<pad> The wet sand was easy to grasp.</s>
tensor(-0.2707)
<pad> The sand was hard to grasp.</s><pad><pad>
tensor(-0.3358)
<pad> The dry sand was difficult to grasp.</s><pad>
tensor(-0.4160)


In [4]:
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-j-6B", cache_dir=hfcache)

Downloading:   0%|          | 0.00/836 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/11.3G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/619 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/779k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.31M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.94k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/357 [00:00<?, ?B/s]

In [9]:
prompt = (
    "In a shocking finding, scientists discovered a herd of unicorns living in a remote, "
    "previously unexplored valley, in the Andes Mountains. Even more surprising to the "
    "researchers was the fact that the unicorns spoke perfect English."
)
input_ids = tokenizer(prompt, return_tensors="pt").input_ids
gen_tokens = model.generate(
    input_ids,
    do_sample=True,
    temperature=0.9,
    max_length=100,
)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


RuntimeError: "LayerNormKernelImpl" not implemented for 'Half'